# <h1><center>Spatial Data Science</center></h1>
## <h1><center>Interactive data visualization of analyzed data on map view</center></h1>

#### Importing following libraries for rendering a proximity analysis between old and new sites data analyzed data 

In [2]:
import geopandas as gpd
import folium
import pandas as pd
import os

In [3]:
import random
import string

#### Reading an existing analyzed shape file data using geopandas read_file function to render it using Folium

In [4]:
old_sites = gpd.read_file("data/New_Sites-old_fibre_sites/New_Sites-old_fibre_sites_Nearest_To.shp")

In [5]:
new_sites = gpd.read_file("data/New_Sites-old_fibre_sites/New_Sites-old_fibre_sites_Nearest_From.shp")

In [6]:
nearest_distance = gpd.read_file("data/New_Sites-old_fibre_sites/New_Sites-old_fibre_sites_Nearest_Connection.shp")

In [7]:
nearest_distance.shape

(1200, 10)

#### Correcting Column in the dataframes

In [8]:
nearest_distance=nearest_distance.rename(columns={'NEAREST_DI':'Distance in meters: '})

In [9]:
new_sites = new_sites.rename(columns={'SITE_NAME':'NEW_SITE_NAME'})
new_sites.shape


(1200, 10)

In [10]:
new_sites=new_sites.drop(columns=['LRP_RAN_SI','EXCEL_ROW_'])

In [11]:
new_sites.head()
new_sites.shape

(1200, 8)

In [12]:
old_sites = old_sites.rename(columns={'LRP_RAN_SI':'OLD_SITES'})
old_sites.shape

(1200, 10)

In [13]:
old_sites=old_sites.drop(columns=['SITE_NAME','SITE_ID','REGION','LAT','LONG','EXCEL_ROW_'])

In [14]:
old_sites.head()
old_sites.shape

(1200, 4)

#### We will mask user data by replacing the values with a random string generator function

In [15]:
N=7
res= ''.join(random.choices(string.ascii_uppercase+string.digits,k=N))
res

'EYY1D19'

In [22]:
  def gen_str(N=7,char=string.ascii_uppercase+string.digits):
        return ''.join(random.choice(char) for i in range(N))
   



In [19]:
new_sites.replace(to_replace=new_sites['NEW_SITE_NAME'].values,value=gen_str(),inplace=True)
new_sites.replace(to_replace=new_sites['SITE_ID'].values,value=gen_str(),inplace=True)

In [20]:
old_sites.replace(to_replace=old_sites['OLD_SITES'].values,value=gen_str(),inplace=True)

In [21]:
new_sites.head()

,SITE_ID,REGION,NEW_SITE_NAME,NEAREST_DI,LAT,LONG,NEAREST_TY,geometry
0,TRNGTF9,SGS,L423GOF,446.05197,-26.23449,28.02534,Direct,POINT (28.02534 -26.23449)
1,TRNGTF9,SGS,L423GOF,498.81512,-26.19752,28.02345,Direct,POINT (28.02345 -26.19752)
2,TRNGTF9,SGS,L423GOF,519.53755,-26.19708,28.02219,Direct,POINT (28.02219 -26.19708)
3,TRNGTF9,SGS,L423GOF,420.77907,-26.20883,28.01666,Direct,POINT (28.01666 -26.20883)
4,TRNGTF9,SGS,L423GOF,435.51151,-26.20126,28.00910,Direct,POINT (28.00910 -26.20126)


In [28]:
old_sites.head()


,NEAREST_DI,OLD_SITES,NEAREST_TY,geometry
0,446.05197,EBD5MJP,Direct,POINT (28.02111 -26.23321)
1,498.81512,EBD5MJP,Direct,POINT (28.02174 -26.20175)
2,519.53755,EBD5MJP,Direct,POINT (28.02174 -26.20175)
3,420.77907,EBD5MJP,Direct,POINT (28.02084 -26.20926)
4,435.51151,EBD5MJP,Direct,POINT (28.01181 -26.20434)


#### Setting map location as per requirement for example South Africa 

In [23]:
m = folium.Map(location= [-28.4792625,24.6727135],zoom_start=6)
m

####  Visualization of data on Map

In [24]:
# Lets render new site data as green circles on map
labels_new = list(new_sites.NEW_SITE_NAME)
x_coord = list(new_sites.LONG)
y_coord = list(new_sites.LAT)
m = folium.Map([-28.4792625,24.6727135], zoom_start=5, tiles='cartodbpositron')
for lat,lon,labels_new in zip(y_coord,x_coord,labels_new):
    folium.Circle(location=[lat,lon],color='green', radius=0.19,popup=labels_new).add_to(m)
m


In [25]:
# Easy Visualization of dataframe directly using folium.GeoJson for line geometry data 

m = folium.Map([-28.4792625,24.6727135], zoom_start=5, tiles='cartodbpositron')

folium.GeoJson(nearest_distance.sample(1000),popup=folium.GeoJsonPopup(fields=['Distance in meters: '])).add_to(m)
m


#### Data visualization on an interactive map view showing data with useful information in tooltip,color and location of old and new sites and a shortest distance between them within 1200 meters buffer area

In [26]:
m = folium.Map([-28.4792625,24.6727135], zoom_start=5, tiles='Stamen Terrain')
#We want to add site name as a tooltip so for this we will create a list as shown below
labels_new = list(new_sites.NEW_SITE_NAME)
x_coord = list(new_sites.LONG)
y_coord = list(new_sites.LAT)

for lat,lon,labels_new in zip(y_coord,x_coord,labels_new):
    folium.Circle(location=[lat,lon],color='green',fill_color='green', radius=100,popup=labels_new,tooltip=labels_new).add_to(m)

labels_old = list(old_sites.OLD_SITES)
x_coord = list(old_sites.geometry.x)
y_coord = list(old_sites.geometry.y)

for lat,lon,labels_old in zip(y_coord,x_coord,labels_old):
    folium.Circle(location=[lat,lon],color='red',fill_color='red', radius=100,popup=labels_old,tooltip=labels_old).add_to(m)
#To style line geomtery we need to create a dictionary and use style fucntion lamda,also GeoJsonTooltip provides a way to add tooltip for the line geometry
style = {'fillColor': 'black', 'color': 'black'}
folium.GeoJson(nearest_distance,popup=folium.GeoJsonPopup(fields=['Distance in meters: ']),style_function=lambda x: style,tooltip=folium.GeoJsonTooltip(fields=['Distance in meters: '])).add_to(m)

folium.TileLayer('openstreetmap').add_to(m)

folium.LayerControl().add_to(m)

m.save('data_analysis_map.html')

#### Fast Marker Cluster with distinguish markers for new and old sites data

In [27]:

x1_coord = list(old_sites.geometry.x)
y1_coord = list(old_sites.geometry.y)

from folium.plugins import FastMarkerCluster

m1 = folium.Map(location=[-28.4792625,24.6727135],
                        zoom_start=2,
                        tiles='CartoDB dark_matter')


#need to call a callback function within FastMarkerCluster to change icon related properties. In below example old site is shown
#in red color marker and new site in green

callback_new = """\
function (row) {
var icon, marker;
icon = L.AwesomeMarkers.icon({
icon: "map-marker", markerColor: "green"});
marker = L.marker(new L.LatLng(row[0], row[1]));
marker.setIcon(icon);
return marker;
};
"""
callback_old = """\
function (row) {
var icon, marker;
icon = L.AwesomeMarkers.icon({
icon: "map-marker", markerColor: "red"});
marker = L.marker(new L.LatLng(row[0], row[1]));
marker.setIcon(icon);
return marker;
};
"""
FastMarkerCluster(new_sites[['LAT','LONG']].values.tolist(),callback=callback_new).add_to(m1)
FastMarkerCluster(zip(y1_coord,x1_coord),callback=callback_old).add_to(m1)

 
folium.TileLayer('openstreetmap').add_to(m1)
folium.TileLayer('cartodbpositron').add_to(m1)

folium.LayerControl().add_to(m1)

m1.save('data_analysis_map_cluster.html')